<a href="https://colab.research.google.com/github/Artikasaridewi/data/blob/main/Next_Ionogram_Frame_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Description: 

Oleh: Varuliantor Dear, Annis Siradj M, Artika, dan Yana Robiana

Next Ionogram Plot Prediction versi 2 GRAY IMAGE

Sumber: https://keras.io/examples/vision/conv_lstm/#dataset-construction  dengan sedikit modifikasi seperti import model dan lain lain

Input: Dataset berupa frame ionogram per 15 menit dallam satu hari selama satu bulan

Output: Next Frame Ionogram

Target tahun 2022

# IMPORT DATA FROM DRIVE

In [ ]:
import numpy as np
import os
from matplotlib import pyplot as plt
import cv2
import random


# IMPORT DATA FROM GITHUB

In [ ]:
!git clone https://github.com/varuliant/Ionogram_KOE.git  

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import zipfile
import os

In [ ]:
!wget --no-check-certificate \
    "https://github.com/varuliant/Ionogram_KOE.git" \
    -O "/tmp/cats-and-dogs.zip"


zip_ref = zipfile.ZipFile('/tmp/cats-and-dogs.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall('/tmp') #Extracts the files into the /tmp folder
zip_ref.close()

In [ ]:
# Test data apakah tetrsimpan
import os
from google.colab import drive
import cv2
import matplotlib.pyplot as plt

# Path Data
path = "/content/Ionogram_KOE/Clean/220401KO/"   # Gunakan tanda quote 2 

#print(os.listdir(path))
image = cv2.imread(path + os.listdir(path)[0])
type(image)
#print(os.listdir(path)[0])
#print(image)
plt.imshow(image)

In [ ]:
print(image.shape)

# GENERATE Dataset dari Folder dan Sub Folder sendiri
Terdiri dari:
1. Fungsi Membaca Image atau frame pada Sub folder dan mengubahnya kedalam array 4D
2. Iterasi per sub folder dari main folder sehingga menjadi array 5 D
yang dikelompokkan kedalam sub folder 

Syarat: Jumlah frame dalam sub folder harus sama. Contoh Ionogram 1 hari 96 image

Opsi untuk performa:
1. Croping wilayah jejak ionogram tanpa axis secara konsisten
2. Grayscale image
3. Clean ionogram traces

In [ ]:
from os import listdir
from os.path import isdir
import cv2
import numpy as np

## MENYIAPKAN DATA SET DARI FOLDER PER HARI (FINAL)

In [ ]:
# Fungsi ini dipilih karena ada fungsi sort nama file agar berurutan
def ambilfile2(sub_dir):
  y=70
  x=110
  h=380
  w=400
  width = 80
  height = 80
  images_sub = np.empty(shape=(0, height, width,1), dtype='uint8') 
  #images_sub=[]
  names = os.listdir(sub_dir)
  names.sort()
  for filename in names:
    image=cv2.imread(sub_dir + '/' + filename)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # image langsung sebagai gray
    #image = cv2.bitwise_not(image)  # invert the B n W to be Black Background
    # bisa juga gunakan hanya merah saja agar jejak ionogram jelas # split the image into its three channels
    # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # (R,G,B) = cv2.split(image)
    #image = R
    # UNTUK GRAY SCALE background Black ###################
    #se = cv2.getStructuringElement(cv2.MORPH_RECT , (8,8))
    #bg = cv2.morphologyEx(image, cv2.MORPH_DILATE, se)
    #image = cv2.divide(image, bg, scale=255)
    #image = cv2.threshold(image, 127, 255, cv2.THRESH_OTSU )[1] 
    #(thresh, blackAndWhiteImage) = cv2.threshold(grayImage, 127, 255, cv2.THRESH_BINARY)
    #######################################
    image = image[y:y+h, x:x+w] # Trik untuk croping wilayah ionogram tertentu
    image = cv2.resize(image, (height, width))
    image = image[..., np.newaxis]#tambahkan extra array agar sesuai dengan feeder model
    #print (image.shape)
    images_sub = np.append(images_sub, [image/255], axis=0) # ini dat per sub folder
  return images_sub 

#Untuk test panggil
#subdirr="/content/Ionogram_KOE/Clean/220401KO/"
#dataset_frame=ambilfile(subdirr)
#print (dataset_frame.shape)

In [ ]:
import os
from os import listdir
from os.path import isdir
import cv2
import numpy as np
# LOPPING UTAMA UNTUK PREPARING DATASET DARI FOLDER UTAMA
#main_dir="/content/Ionogram_KOE/Original/"
main_dir="/content/Ionogram_KOE/Clean/"


dataframe = [] # empty list of dataframe
for subdir in listdir(main_dir):
  #print(subdir)
  path_sub_dir=main_dir + subdir
  #clling sub dir
  subfolder_frame=ambilfile2(path_sub_dir)
  
  if not isdir(main_dir):
    print(path_sub_dir)
    continue
  # subfolder_frame = subfolder_frame[np.newaxis, ...] # buat menjadi array dimensi baru
  # Tumpuk menjadi array dimensi baru dari subfolder_frame
  # dataset_frame = np.append(dataset_frame, subfolder_frame, axis=0) # ini harus di deklarasikan dulu shape awalnya dan harus sama ukurannya
  list_sub = subfolder_frame.tolist() # ubah kedalam list dari array nupy
  dataframe.append(list_sub)
dataset_frame = np.asarray(dataframe)
  
# OK Syarat Data frame per subfolder harus memiliki jumlah yang sama agar tidak eror  

In [ ]:
print(listdir(main_dir))

In [ ]:
# Periksa susunan dimensi array dataset per frame
print (subfolder_frame.shape)
print (len(list_sub))
print (len(dataframe))
print (dataset_frame.shape)

DEFINISI ULANG JUMLAH DATFRAME SHAPE AGAR TIDAK CRASH AKIBAT OVERLOAD RAM

In [ ]:
# Definisi ulang dataset shape dari 96 image menajdi 24 image
dataset_frame=dataset_frame[:, 24:48, :, :]
print(dataset_frame.shape)

# MENYUSUN DATA SET UNTUK TRAINING DAN VALIDASI

In [ ]:
# Split into train and validation sets using indexing to optimize memory.
indexes = np.arange(dataset_frame.shape[0])
np.random.shuffle(indexes)
train_index = indexes[: int(0.9 * dataset_frame.shape[0])]
val_index = indexes[int(0.9 * dataset_frame.shape[0]) :]
train_dataset = dataset_frame[train_index]
val_dataset = dataset_frame[val_index]


# Normalize the data to the 0-1 range untuk GRAY SCALE
#train_dataset = train_dataset / 255
#val_dataset = val_dataset / 255

# We'll define a helper function to shift the frames, where
# `x` is frames 0 to n - 1, and `y` is frames 1 to n.
def create_shifted_frames(data):
    x = data[:, 0 : data.shape[1] - 1, :, :]
    y = data[:, 1 : data.shape[1], :, :]
    return x, y



In [ ]:
print(dataset_frame.shape)

In [ ]:
# Apply the processing function to the datasets.
x_train, y_train = create_shifted_frames(train_dataset)
x_val, y_val = create_shifted_frames(val_dataset)

# Inspect the dataset.
print("Training Dataset Shapes: " + str(x_train.shape) + ", " + str(y_train.shape))
print("Validation Dataset Shapes: " + str(x_val.shape) + ", " + str(y_val.shape))

## Memeriksa Dataset Frame yang telah disusun

In [ ]:
import matplotlib.pyplot as plt

indexes = np.arange(dataset_frame.shape[0])

# Construct a figure on which we will visualize the images.
fig, axes = plt.subplots(4, 6, figsize=(10, 8))

# Plot each of the sequential images for one random data example.
data_choice = np.random.choice(range(len(train_dataset)), size=1)[0]
for idx, ax in enumerate(axes.flat):
    ax.imshow(np.squeeze(train_dataset[data_choice][idx]),cmap='gray') # tambahan cmap-'gray' agar tampil seperti plot yang benar
    ax.set_title(f"Frame {idx + 1}")
    ax.axis("off")

# Print information and display the figure.
print(f"Displaying frames for example {data_choice}.")
plt.show()

# KONSTRUKSI MODEL 

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


In [ ]:
# Construct the input layer with no definite frame size.
inp = layers.Input(shape=(None, *x_train.shape[2:]))

# We will construct 3 `ConvLSTM2D` layers with batch normalization,
# followed by a `Conv3D` layer for the spatiotemporal outputs.
x = layers.ConvLSTM2D(filters=64, kernel_size=(5, 5), padding="same", return_sequences=True, activation="relu",)(inp)
x = layers.BatchNormalization()(x)
x = layers.ConvLSTM2D(    filters=64,    kernel_size=(3, 3),    padding="same",    return_sequences=True, activation="relu",)(x)
#x = layers.UpSampling2D((2,2))(x)
x = layers.BatchNormalization()(x)
x = layers.ConvLSTM2D( filters=64, kernel_size=(1, 1), padding="same", return_sequences=True, activation="relu",)(x)
x = layers.Conv3D( filters=1, kernel_size=(3, 3, 3), activation="sigmoid", padding="same")(x)  # original filters adalah1 karena untuk gray sclale. kita ubah 3

# Next, we will build the complete model and compile it.
model = keras.models.Model(inp, x)
model.compile(loss=keras.losses.binary_crossentropy, optimizer=keras.optimizers.Adam(),)

In [ ]:
print(model.summary())

# TRAINING MODEL

In [ ]:
# Define some callbacks to improve training.
early_stopping = keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=5)

# Define modifiable training hyperparameters.
epochs = 1000
batch_size = 5

# Fit the model to the training data.
history=model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_val, y_val), callbacks=[early_stopping, reduce_lr],)

In [ ]:
# Simpan hasil model training yang dilakukan
# sumber: https://www.tensorflow.org/guide/keras/save_and_serialize
# Calling `save('my_model.h5')` creates a h5 file `my_model.h5`.
model.save("my_h5_model_1gray.h5")

In [ ]:
# Panggil model
filepath = 'my_h5_model_1gray.h5'

In [ ]:
history_dict = history.history 
print(history_dict.keys())

In [ ]:
import matplotlib.pyplot as plt
plt.subplot(2, 1, 1)
plt.plot(history.history['val_loss'])
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()

plt.subplot(2, 1, 2)
plt.plot(history.history['loss'])
plt.xlabel('Epochs')
plt.ylabel('LOSS')
plt.show()

# PREDIKSI NEXT FRAME DARI DATASET YANG DIPILIH SECARA ACAK

In [ ]:
# Select a random example from the validation dataset.
example = val_dataset[np.random.choice(range(len(val_dataset)), size=1)[0]]

# Pick the first/last ten frames from the example.
frames = example[:12, ...]
original_frames = example[12:, ...]

# Predict a new set of 12 frames.
for _ in range(12):
    # Extract the model's prediction and post-process it.
    new_prediction = model.predict(np.expand_dims(frames, axis=0))
    new_prediction = np.squeeze(new_prediction, axis=0)
    predicted_frame = np.expand_dims(new_prediction[-1, ...], axis=0)

    # Extend the set of prediction frames.
    frames = np.concatenate((frames, predicted_frame), axis=0)

# Construct a figure for the original and new frames.
fig, axes = plt.subplots(2, 12, figsize=(24, 4))

# Plot the original frames.
for idx, ax in enumerate(axes[0]):
    ax.imshow(np.squeeze(original_frames[idx]),cmap='gray')
    ax.set_title(f"Frame {idx + 13}")
    ax.axis("off")

# Plot the new frames.
new_frames = frames[12:, ...]
for idx, ax in enumerate(axes[1]):
    ax.imshow(np.squeeze(new_frames[idx]),cmap='gray')
    ax.set_title(f"Frame {idx + 13}")
    ax.axis("off")

# Display the figure.
plt.show()

In [ ]:
# PERIKSA GAMBAR HASIL PREDIKSI SECARA MANUAL 
predicted_frame.shape
plt.imshow(np.squeeze(predicted_frame),cmap='gray')
#plt.show()


# BUAT VIDEO FRAME BY FRAME 

In [ ]:
import io
import imageio
from IPython.display import Image, display
from ipywidgets import widgets, Layout, HBox

In [ ]:
# Select a few random examples from the dataset.
examples = val_dataset[np.random.choice(range(len(val_dataset)), size=5)]

# Iterate over the examples and predict the frames.
predicted_videos = []
for example in examples:
    # Pick the first/last ten frames from the example.
    frames = example[:12, ...]
    original_frames = example[12:, ...]
    new_predictions = np.zeros(shape=(12, *frames[0].shape))

    # Predict a new set of 12 frames.
    for i in range(12):
        # Extract the model's prediction and post-process it.
        frames = example[: 12 + i + 1, ...]
        new_prediction = model.predict(np.expand_dims(frames, axis=0))
        new_prediction = np.squeeze(new_prediction, axis=0)
        predicted_frame = np.expand_dims(new_prediction[-1, ...], axis=0)

        # Extend the set of prediction frames.
        new_predictions[i] = predicted_frame

    # Create and save GIFs for each of the ground truth/prediction images.
    for frame_set in [original_frames, new_predictions]:
        # Construct a GIF from the selected video frames.
        current_frames = np.squeeze(frame_set)
        current_frames = current_frames[..., np.newaxis] * np.ones(3)
        current_frames = (current_frames * 255).astype(np.uint8)
        current_frames = list(current_frames)

        # Construct a GIF from the frames.
        with io.BytesIO() as gif:
            imageio.mimsave(gif, current_frames, "GIF", fps=5)
            predicted_videos.append(gif.getvalue())

# Display the videos.
print(" Truth\tPrediction")
for i in range(0, len(predicted_videos), 2):
    # Construct and display an `HBox` with the ground truth and prediction.
    box = HBox(
        [
            widgets.Image(value=predicted_videos[i]),
            widgets.Image(value=predicted_videos[i + 1]),
        ]
    )
    display(box)

# Reset Colab untuk pengujian berikutnya 

In [ ]:
import os, signal
os.kill(os.getpid(), signal.SIGKILL)